In [3]:
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from stk_actor.agent import UnifiedSACPolicy
from stk_actor.wrappers import PreprocessObservationWrapper
import torch

from pystk2_gymnasium import AgentSpec

vec_env = make_vec_env("supertuxkart/flattened_multidiscrete-v0", seed=0,
                        # wrapper_class=lambda x : ObsTimeExtensionWrapper(PreprocessObservationWrapper(x)), 
                        wrapper_class=lambda x : (PreprocessObservationWrapper(x)), 
                       n_envs=1, env_kwargs={
    'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"),  #'track':'abyss', #'num_kart':2, 'difficulty':0
})

# net_arch=[512,512,512]
# activation_fn=torch.nn.SiLU
# filename = 'policy_512_512_512_SiLU_2_statedict'

net_arch=[512,512,512,512]
activation_fn=torch.nn.SiLU
filename = 'policy_512_512_512_512_SiLU_3_statedict'

model = PPO(
        "MlpPolicy", 
        vec_env, 
        verbose=1, 
        policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,), 
        device='cpu',
        n_steps=5,
        tensorboard_log="./outputs/"
)
action_dims = [space.n for space in vec_env.action_space]
unified_policy = UnifiedSACPolicy(
    vec_env.observation_space, 
    action_dims, 
    net_arch=net_arch, 
    activation_fn=activation_fn
)
unified_policy.load_state_dict(torch.load(filename))
model.policy.load_state_dict(unified_policy.shared.state_dict())
vec_env.close()
# model = PPO.load("ppti_ppo_1500", custom_objects={'policy_kwargs' :  dict(net_arch=net_arch, activation_fn=activation_fn), })
model = PPO.load("ppti_ppo_1500_batch128_clip0001_ent0001", custom_objects={'policy_kwargs' :  dict(net_arch=net_arch, activation_fn=activation_fn), })


..:: Antarctica Rendering Engine 2.0 ::..
Using cpu device


/opt/homebrew/Caskroom/miniforge/base/envs/bbrl/lib/python3.10/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 5`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 5
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=5 and n_envs=1)
  warnings.warn(
/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_40392/3916416425.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be ex

In [ ]:
tracks = [
    # 'black_forest',
    # 'olivermath',
    # 'minigolf',
    # 'gran_paradiso_island',
    # 'candela_city',
    # 'fortmagma'
    # 'ravenbridge_mansion',
    # 'mines',
    # 'snowmountain',
    #  'abyss',
    # 'cocoa_temple',
    # 'cornfield_crossing',
    # 'hacienda',
    # 'lighthouse',
    # 'sandtrack',
    # 'scotland',
    # 'snowtuxpeak',
    # 'stk_enterprise',
    # 'volcano_island',
    # 'xr591',
    # 'zengarden'
]
# for track in tracks:
while True:
    vec_env = make_vec_env("supertuxkart/flattened_multidiscrete-v0", #seed=0,
                        # wrapper_class=lambda x : ObsTimeExtensionWrapper(PreprocessObservationWrapper(x)), 
                        wrapper_class=lambda x : (PreprocessObservationWrapper(x, ret_dict=False)), 
                       n_envs=1, env_kwargs={
    'render_mode':"human", 'agent':AgentSpec(use_ai=False, name="walid"), 'laps':2,#'track':track, #'num_kart':2, 'difficulty':0
    })
    obs = vec_env.reset()
    while True:
        ix = 0
        action, _states = model.predict(obs, deterministic=False)
        obs, rewards, dones, info = vec_env.step(action)
        vec_env.render("human")
        ix += 1
        if ix == 256:
            break
    vec_env.close()



..:: Antarctica Rendering Engine 2.0 ::..


2025-01-17 11:17:43.965 python[40338:1599379] +[IMKClient subclass]: chose IMKClient_Modern
2025-01-17 11:17:43.965 python[40338:1599379] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [4]:
import torch
torch.save( model.policy.state_dict(), 'ppo_policy_512_512_512_512_SiLU_4_statedict')